In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
from strategy_v2.Strategy import *
from strategy_v2.TradingSubSystem import *
from strategy_v2.Portfolio import *
from strategy_v2.TransactionCost import *
from strategy_v2.Executor import *
from utils.data_helper import *
from utils.data import *
from utils.performance import *
import cvxpy as cp
from plotly.subplots import make_subplots
from croniter import croniter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Portfolio General Settings
We assume the code is run at T before market open, so it all data on and before T-1 close. Therefore, the end_date should be today - BDay(1)

In [3]:
end_date = get_today()
start_date = pd.to_datetime(datetime(2022, 1, 3))
vol_target = 0.5
max_leverage = 1

# Linear Optimization (Risk-Adjusted Return)

- How to determine the capital?

Difficulty here is portfolio might use different leverage per rebalance, looking at the asset MV might not give you the total portfolio capital (e.g. 70% leverage, your stocks only worth 70% of your capital now)

we can use the market value on Futu divides by the leverage used for last periods. This will give you the total captial for the portfolio

Then we use this capital for next rebalance

We shuold strategically fix this issue by reading capital from Futu or keep track the capital ourselves

In [12]:
portfolio = PortfolioLinearOpt(
    capital=188912.2609673791,    
    name='ETF Buy and Hold Portfolio',
    lookback_period=60,
    opt_freq=1,
    rebalance_iter=RebalancerIter('0 0 * * Fri', 2),    
    tc_model=TransactionCostFutu(),    
    systems=[
        TradingSubSystemSingle(vol_target=vol_target, instruments=['META'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['TSLA'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['NVDA'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['AAPL'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['EWY'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['DXJ'],   strategy=[BuyAndHoldStrategy(confidence=2)], max_leverage=max_leverage),        
        #TradingSubSystemSingle(vol_target=vol_target, instruments=['VOE'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        #TradingSubSystemSingle(vol_target=vol_target, instruments=['VBR'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['BRK-B'], strategy=[BuyAndHoldStrategy(confidence=2)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['SPY'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['QQQ'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),        
])

portfolio.set_start_date(start_date)
portfolio.set_end_date(end_date)
portfolio.backtest_subsystems()
portfolio.optimize(gamma=10, hhi=0.2)
portfolio.backtest()
portfolio.rebalance()
portfolio.performance(show_all_rets=True)

2024-03-09 00:28:31,966 - TradingSubSystemSingle [META] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-03-08......
2024-03-09 00:28:31,983 - TradingSubSystemSingle [META] - INFO - Volatility Target = 50.0% | Price Volatility = 27.8% | Last Scale Factor = 1.80
2024-03-09 00:28:31,992 - TradingSubSystemSingle [TSLA] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-03-08......
2024-03-09 00:28:32,005 - TradingSubSystemSingle [TSLA] - INFO - Volatility Target = 50.0% | Price Volatility = 47.8% | Last Scale Factor = 1.05
2024-03-09 00:28:32,014 - TradingSubSystemSingle [NVDA] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-03-08......
2024-03-09 00:28:32,023 - TradingSubSystemSingle [NVDA] - INFO - Volatility Target = 50.0% | Price Volatility = 67.3% | Last Scale Factor = 0.74
2024-03-09 00:28:32,028 - TradingSubSystemSingle [AAPL] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-03-08

,Rebalanced Portfolio,Optimized Portfolio,^SPX,Single - META (BAH1),Single - TSLA (BAH1),Single - NVDA (BAH1),Single - AAPL (BAH1),Single - EWY (BAH1),Single - DXJ (BAH2),Single - BRK-B (BAH2),Single - SPY (BAH1),Single - QQQ (BAH1)
Measure,,,,,,,,,,,,
Cumulative Return,1.670138,1.673024,1.076749,1.620055,0.532178,3.177905,0.978148,0.888364,1.803337,1.345719,1.119639,1.131936
Annualized Return,0.259921,0.260527,0.051892,0.350792,-0.166754,0.650508,0.030074,-0.023741,0.284678,0.153158,0.069935,0.089310
Annualized Volatility,0.219706,0.218779,0.189059,0.502225,0.494747,0.492125,0.283901,0.247929,0.163675,0.182240,0.189472,0.254140
Annualized Sharpe Ratio,1.022893,1.030001,0.088368,0.628417,-0.408167,1.250337,-0.018003,-0.237672,1.524318,0.647348,0.183402,0.212974
Maximum Drawdown,-0.262085,-0.270334,-0.254251,-0.682479,-0.635372,-0.531787,-0.309294,-0.406293,-0.117846,-0.265790,-0.244964,-0.348280


# Equal Weighted Portfolios

In [13]:
# portfolio = PortfolioStandard(    
#     capital=200000,
#     name='ETF Buy and Hold Portfolio',    
#     rebalance_iter=RebalancerIter('0 0 * * Fri', 2),
#     tc_model=TransactionCostFutu(),    
#     systems=[
#         # TradingSubSystemSingle(vol_target=vol_target, instruments=['META'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         # TradingSubSystemSingle(vol_target=vol_target, instruments=['TSLA'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         # TradingSubSystemSingle(vol_target=vol_target, instruments=['NVDA'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         # TradingSubSystemSingle(vol_target=vol_target, instruments=['AAPL'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['EWY'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['DXJ'],   strategy=[BuyAndHoldStrategy(confidence=2)], max_leverage=max_leverage),        
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['VOE'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['VBR'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['BRK-B'], strategy=[BuyAndHoldStrategy(confidence=2)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['SPY'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['QQQ'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),       
# ])

# portfolio.set_start_date(start_date)
# portfolio.set_end_date(end_date)
# portfolio.backtest_subsystems()
# portfolio.optimize()
# portfolio.backtest()
# portfolio.rebalance()
# portfolio.performance()

# Execute the Portfolio

In [16]:
executor = ExecutorFutu(is_test=False)
executor.set_portfolio(portfolio)
orders = executor.execute(px_interval='5m')
orders

2024-03-09 00:36:46,826 - ExecutorFutu - INFO - market: US
2024-03-09 00:36:46,828 - ExecutorFutu - INFO - Cancel all orders first before executing.....


2024-03-09 00:36:46,840 | 86811 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=21, host=127.0.0.1, port=11111, user_id=18214795
2024-03-09 00:36:47,000 | 86811 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=21


2024-03-09 00:36:50,005 - Linear Optimized Portfolio (ETF Buy and Hold Portfolio) - CRITICAL - Portfolio target capital is not specified, use initial backtest capital of $188,912
2024-03-09 00:36:50,005 - Linear Optimized Portfolio (ETF Buy and Hold Portfolio) - INFO - Generate trade position based on target capital of $188,912
2024-03-09 00:36:50,009 - ExecutorFutu - INFO - Execute Linear Optimized Portfolio (ETF Buy and Hold Portfolio) position based on 2024-03-08


2024-03-09 00:36:50,012 | 86811 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=22, host=127.0.0.1, port=11111, user_id=18214795


2024-03-09 00:36:50,024 - Futu - INFO - 7 Positions: US.SPY, US.QQQ, US.NVDA, US.META, US.EWY, US.DXJ, US.BRK.B


2024-03-09 00:36:50,026 | 86811 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=22


,instrument,target,current,turnover
0,META,62.0,62.0,0.0
1,TSLA,0.0,0.0,0.0
2,NVDA,47.0,47.0,0.0
3,AAPL,0.0,0.0,0.0
4,EWY,109.0,109.0,0.0
5,DXJ,378.0,378.0,0.0
6,BRK-B,67.0,67.0,0.0
7,SPY,30.0,30.0,0.0
8,QQQ,21.0,21.0,0.0


[*********************100%***********************]  9 of 9 completed


2024-03-09 00:36:50,524 - ExecutorFutu - INFO - getting last 5m prices since 2024-03-08 11:35:00 for order limit price


""
